In [1]:
import os
import sys
import gzip
import code
import urllib
import importlib
import tensorflow.python.platform
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm
from PIL import Image

In [2]:
COLAB = False
BRANCH = 'main'

In [3]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/Shareddrives/ML_Road_Segmentation/CS-433-project-ben/project_road_segmentation'
    os.chdir(drive_path)
    from helpers.colab import mount_and_pull
    BRANCH_NAME = BRANCH
    mount_and_pull(BRANCH_NAME, drive, os)

In [4]:
%load_ext autoreload
%autoreload 2

from helpers.image_processing import *
from helpers.file_manipulation import *
from helpers.constants import *
from helpers.prediction_checking import *
from helpers.machine_learning import *
from helpers.loss_functions import *

In [5]:
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
SAVE_MODEL = False
GENERATE_PREDICTION = False  # If True, will generate a CSV to submit on AICrowd

MODEL_NAME = 'unet_2'  # For now, cnn, unet-1, unet-2
PATCH_PREDICTION = False
SAVE_DIR = MODELS_SAVE_DIR + MODEL_NAME + '/'

NUM_EPOCHS = 250

In [6]:
mod = importlib.import_module('models.' + MODEL_NAME)
model_function = getattr(mod, MODEL_NAME)

In [10]:
X_train, X_test, y_train, y_test, validation_set = get_train_test_manual_split()

Loading data/generated/shift/groundtruth/: 100%|████████████████████████████████████| 100/100 [00:00<00:00, 385.58it/s]


In [8]:
if PATCH_PREDICTION:
    from helpers.patch_labeling import *
    old_y_train = y_train
    old_y_test = y_test
    y_train = label_to_patch_prediction(y_train)
    y_test = label_to_patch_prediction(y_test)
    print(y_train.shape)
    print(y_test.shape)
    visualize_patches(old_y_train, y_train, number=3, resize=False)

In [ ]:
# TODO : Try to train unet-2 with 250 epoch and focal tversky loss and check for overfit
# Hyperparameters to tweak : Alpha and Gamma from focal tversky loss for accuracy, and dropout rate for overfit
metrics=[tf.keras.metrics.Precision(),
         tf.keras.metrics.Recall(),
        ]
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5)
            ]
if RESTORE_MODEL:
    model = tf.keras.models.load_model(SAVE_DIR, custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
else:  
    model = model_function()
    model.compile(optimizer='adam',
                loss=dice_loss,
                metrics=metrics)
    history = model.fit(X_train, y_train, 
                        epochs = NUM_EPOCHS,
                        validation_data=validation_set,
                        callbacks=callbacks)

Epoch 1/250
1/4 [======>.......................] - ETA: 0s - loss: 0.6915 - precision_1: 0.2233 - recall_1: 0.7384

In [ ]:
if SAVE_MODEL:
    model.save(SAVE_DIR)

In [19]:
train_predictions = model.predict(X_train).squeeze()
test_predictions = model.predict(X_test).squeeze()

NameError: name 'model' is not defined

In [ ]:
if PATCH_PREDICTION:
    train_predictions = resize_images(train_predictions, TRAINING_IMG_SIZE)
    test_predictions = resize_images(test_predictions, TRAINING_IMG_SIZE)

In [ ]:
visualize_random_predictions(X_train, y_train, train_predictions, size=10)

In [ ]:
visualize_random_predictions(X_test, y_test, test_predictions, size=10)

In [ ]:
compute_entire_images_metrics(y_test, test_predictions)

In [ ]:
if COLAB:
    from helpers.colab import download_model
    from google.colab import files
    download_model(MODEL_NAME, SAVE_DIR, files)